# Проект по оптимизации сортировок

Студент: Алимпиев Дмитрий Алексеевич, ПМ-2001

---



### Установка необходимых модулей и подключение отображения времени и Cython'а

In [ ]:
!pip install ipython-autotime

%load_ext cython
%load_ext autotime

from random import random
import numpy as np

#### **Сортировка методом пузырька на чистом Python'е и при помощи Cython'a**

##### Генерация исходных данных

In [ ]:
def generate_data(length: int) -> list:
  return [random() for _ in range(length)]

##### Функции сортировки

In [ ]:
def python_bubble_sort(array: list):
  n = len(array)

  for i in range(n - 1):
    for j in range(n - i - 1):
        if array[j] > array[j + 1]:
            array[j], array[j + 1] = array[j + 1], array[j]

In [ ]:
%%cython
import copy

def cython_bubble_sort(array: list):
  a = copy.deepcopy(array)
  cdef int n = len(a)
  for i in range(n - 1):
    for j in range(n - 1 - i):
        if a[j] > a[j+1]:
            a[j], a[j+1] = a[j+1], a[j]

##### Запуск и проверка результатов

In [21]:
python_bubble_sort(generate_data(10000))

time: 9.97 s (started: 2022-05-05 22:36:34 +00:00)


In [22]:
cython_bubble_sort(generate_data(10000))

time: 4.57 s (started: 2022-05-05 22:36:44 +00:00)


*Как можно заметить, даже в таком примитивном действии, как сортировка пузырьком списков, длинны 10000 элементов, видна разница между оптимизированными действиями и нет, причём весьма солидная - в 2 раза, учитывая, что ничего, кроме типа данных одной переменной я не менял вручную.*

## **Перемножение матриц NxN на чистом Python'е, с Numpy и при помощи Cython'а**

##### Генерация исходных данных

In [ ]:
from numpy.matrixlib.defmatrix import matrix
def generate_data(length: int, flag="python") -> list:
  if flag == "python": return [[random() for _ in range(length)] for _ in range(length)]
  elif flag == "numpy": return np.matrix(generate_data(length, "python"))

##### Функции перемножения матриц

In [ ]:
def python_matrix_product(matrix1: list, matrix2: list):
  length = len(matrix1) 
  result_matrix = [[0 for i in range(length)] for i in range(length)]

  for i in range(length):
    for j in range(length):
      for k in range(length):
        result_matrix[i][j] += matrix1[i][k] * matrix2[k][j]

In [ ]:
def numpy_matrix_product(matrix1: np.matrix, matrix2: np.matrix):
  np.dot(matrix1, matrix2)

In [ ]:
%%cython -a
import copy

def cython_matrix_product(m1: list, m2: list):
  matrix1, matrix2 = copy.deepcopy(m1), copy.deepcopy(m2)

  cdef int length = len(matrix1) 
  result_matrix = [[0 for i in range(length)] for i in range(length)]

  for i in range(length):
    for j in range(length):
      for k in range(length):
        result_matrix[i][j] += matrix1[i][k] * matrix2[k][j]

##### Запуск и проверка результатов

In [33]:
python_matrix_product(generate_data(400), generate_data(400))

time: 18.9 s (started: 2022-05-05 22:53:46 +00:00)


In [35]:
numpy_matrix_product(generate_data(400, "numpy"), generate_data(400, "numpy"))

time: 66.5 ms (started: 2022-05-05 22:54:40 +00:00)


In [39]:
cython_matrix_product(generate_data(400), generate_data(400))

time: 10.3 s (started: 2022-05-05 22:58:11 +00:00)


*Как можно заметить, самым медленным является, неудивительно, метод, написанный на чистом **python'е** - ~20 секунд при исходном наборе матриц 400x400. На втором месте реализация на **cython'е** - ~10 секунд при тех же исходных данных. Ну и на первом месте, конечно, **numpy** - меньше 0.1 секунды.*

## **Вычисление определителя матрицы NxN на чистом Python'е и при помощи Numpy**

##### Генерация исходных данных

In [ ]:
from numpy.matrixlib.defmatrix import matrix
def generate_data(length: int, flag="python") -> list:
  if flag == "python": return [[random() for _ in range(length)] for _ in range(length)]
  elif flag == "numpy": return np.matrix(generate_data(length, "python"))

#####Функции вычисления определителя

In [ ]:
def python_determinant(matrix: list, mul: int) -> list:
    width = len(matrix)
    if width == 1: return  matrix[0][0] * mul
    else:
        sign = -1
        answer = 0
        for i in range(width):
            m = []
            for j in range(1, width):
                buff = []
                for k in range(width):
                    if k != i:
                        buff.append(matrix[j][k])
                m.append(buff)
            sign *= -1
            answer = answer + mul * python_determinant(m, sign * matrix[0][i])
    return answer

In [ ]:
def numpy_determinant(matrix: np.matrix):
  np.linalg.det(matrix)

#####Запуск и проверка результатов

In [66]:
python_determinant(generate_data(10), 1)

-0.05079331779229891

time: 12.7 s (started: 2022-05-05 23:20:44 +00:00)


In [74]:
numpy_determinant(generate_data(10, "numpy"))

time: 1.74 ms (started: 2022-05-05 23:23:33 +00:00)


*Как можно заметить, в работе с матрицами Numpy вряд ли кому-то вообще уступит, что не удивительно. Это, на мой взгляд - лучшее решение для оптимизации решения математических задач на языке Python.*

# Заключение

В качестве заключения могу отметить, что в целом подобные результаты и предполагал. Однако всё же каждый может многое подчерпнуть из темы оптимизации своего кода. Главное - правильно выбрать способ оптимизации.